# Using Resnet for chest X-ray Tuberculosis classification

In [1]:
import sys, subprocess
print('Python executable:', sys.executable)
print('Python version:', sys.version)
# Install opendatasets and kaggle into the kernel's interpreter (safe from inside notebook)
subprocess.check_call([sys.executable, '-m', 'pip', 'install', '--upgrade', 'pip'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'opendatasets'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'torch'])
print('Installed opendatasets and kaggle. Restart kernel if needed.')

Python executable: c:\Users\Asus\anaconda3\python.exe
Python version: 3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)]
Installed opendatasets and kaggle. Restart kernel if needed.


In [2]:
# (Optional) Upgrade pip from inside the kernel or use a terminal.
# The previous cell already upgrades pip before installing packages.
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'torchvision'])

0

In [3]:
# Diagnostic: show which Python the kernel is using and whether the stdlib 'cgi' module exists
import sys, importlib
print('Kernel executable:', sys.executable)
print('Kernel python version:', sys.version)
spec = importlib.util.find_spec('cgi')
print('cgi module found:', bool(spec), spec)
# Try importing opendatasets to verify installation (may still fail if running under Python 3.13)
try:
    import opendatasets as od
    print('opendatasets import succeeded, version:', getattr(od, '__version__', 'unknown'))
except Exception as e:
    print('opendatasets import failed:', type(e).__name__, e)

Kernel executable: c:\Users\Asus\anaconda3\python.exe
Kernel python version: 3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)]
cgi module found: True ModuleSpec(name='cgi', loader=<_frozen_importlib_external.SourceFileLoader object at 0x0000017B41764C80>, origin='c:\\Users\\Asus\\anaconda3\\Lib\\cgi.py')
opendatasets import succeeded, version: 0.1.22


### Tải dữ liệu và khởi tạo các thư viện xử lý

- Sử dụng **`opendatasets`** để tải tập dữ liệu từ Kaggle.  
- Import các thư viện phục vụ cho:
  - Xử lý ảnh (`PIL`, `torchvision.transforms`)
  - Chia dữ liệu (`random_split`)
  - Tạo mô hình và huấn luyện (`torch`, `torchvision.models`)
  - Đánh giá hiệu suất (`sklearn.metrics`)
  - Tiến trình huấn luyện (`tqdm`)

- Tải dataset TB Chest X-ray từ Kaggle bằng `opendatasets`
- Import các thư viện cần cho tiền xử lý, chia dữ liệu, mô hình và đánh giá

In [4]:
import os
import opendatasets as od
import os
import random
import argparse
from pathlib import Path
from sklearn.model_selection import train_test_split
import numpy as np
from PIL import Image
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report
import torch
from torch import nn
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm

# Download the dataset
dataset_url = 'https://www.kaggle.com/datasets/tawsifurrahman/tuberculosis-tb-chest-xray-dataset'
od.download(dataset_url)

Skipping, found downloaded files in ".\tuberculosis-tb-chest-xray-dataset" (use force=True to force download)


In [5]:
# Define the data directory
data_dir = r'E:\deep-learning\practice\lab\tuberculosis-tb-chest-xray-dataset'
# Define model checkpoints directory
save_dir = "checkpoints"

## Cấu hình môi trường huấn luyện

- `set_seed()`: giúp đảm bảo **kết quả tái lập (reproducibility)** bằng cách cố định seed ngẫu nhiên.  
- `device`: xác định môi trường tính toán:
  - `cuda` nếu có GPU (tăng tốc độ huấn luyện rất nhiều)
  - `cpu` nếu không có GPU

In [6]:
# ---------------------------
# Reproducibility & device
# ---------------------------
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Hàm đánh giá mô hình

Hàm `evaluate_model()` thực hiện:
- Đưa mô hình sang chế độ **đánh giá (`eval`)** – tắt dropout và batchnorm.  
- Duyệt qua toàn bộ batch trong `DataLoader`, tính toán:
  - **Logits** → xác suất bằng `sigmoid`  
  - **Dự đoán nhị phân** theo ngưỡng 0.5  
- Tính các chỉ số:
  - **Accuracy**: tỷ lệ dự đoán đúng  
  - **ROC-AUC**: khả năng phân biệt giữa hai lớp  
  - **Classification Report**: chi tiết Precision, Recall, F1-score cho từng lớp


In [7]:
# ---------------------------
# Utility: metrics
# ---------------------------
def evaluate_model(model, loader):
    model.eval()
    y_true, y_probs, y_pred = [], [], []
    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(device)
            logits = model(xb).squeeze(-1).cpu()  # [batch]
            probs = torch.sigmoid(logits).numpy()
            preds = (probs >= 0.5).astype(int)
            y_probs.extend(probs.tolist())
            y_pred.extend(preds.tolist())
            y_true.extend(yb.numpy().tolist())
    acc = accuracy_score(y_true, y_pred)
    try:
        auc = roc_auc_score(y_true, y_probs)
    except Exception:
        auc = float("nan")
    cls_report = classification_report(y_true, y_pred, digits=4)
    return acc, auc, cls_report

## Chuẩn bị dữ liệu huấn luyện và kiểm định

### Bước 1: Thiết lập seed để đảm bảo tái lập.  
### Bước 2: Định nghĩa các phép biến đổi (`transforms`)
- `RandomResizedCrop`: cắt ngẫu nhiên ảnh về kích thước 224x224 (chuẩn đầu vào của ResNet50)  
- `RandomHorizontalFlip`: tăng cường dữ liệu (data augmentation)  
- `Normalize`: chuẩn hóa ảnh theo thống kê của ImageNet  

### Bước 3: Chia dữ liệu
- Dùng `ImageFolder` để tự động gán nhãn dựa trên tên thư mục.  
- Chia 80% train và 20% validation.


In [8]:
set_seed(42)

# transforms
tf = transforms.Compose([
      transforms.RandomResizedCrop((224,224)),
      transforms.RandomHorizontalFlip(),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
  ])

full_ds = datasets.ImageFolder(os.path.join(data_dir), transform=tf)
# Store class_to_idx before splitting
class_to_idx = full_ds.class_to_idx

train_len = int(len(full_ds)*0.8)
val_len = len(full_ds) - train_len
train_ds, val_ds = random_split(full_ds, [train_len, val_len], generator=torch.Generator().manual_seed(42))

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=1)
val_loader = DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=1)

## Xây dựng mô hình 

### Kiến trúc:
- **ResNet50** là mô hình CNN sâu gồm 50 lớp, sử dụng **residual connections** giúp tránh hiện tượng biến mất gradient khi huấn luyện mạng sâu.
- Ta **khởi tạo từ đầu (`weights=None`)** thay vì dùng pretrained ImageNet để huấn luyện trực tiếp trên ảnh X-quang.

### Điều chỉnh đầu ra:
- ResNet50 gốc có 1000 đầu ra (cho 1000 lớp ImageNet).  
- Ta thay thế lớp `fc` cuối bằng `nn.Linear(num_ftrs, 1)` để **phân loại nhị phân** (bệnh / không bệnh).  
- Sử dụng **`BCEWithLogitsLoss`** (Binary Cross Entropy with Logits) cho bài toán này.

### Tối ưu hóa:
- `Adam` optimizer với learning rate 1e-4.  
- `ReduceLROnPlateau` giảm LR nếu mô hình không cải thiện trong vài epoch.


In [9]:
# model:resnet50 -> single logit output
# using pretrained with weights='DEFAULT'
# training from scratch with weights=None
model = models.resnet50(weights=None)

for param in model.parameters():
      param.requires_grad = True  # fine-tune all (or set False to freeze)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)  # single logit
model = model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=2)

best_auc = 0.0
os.makedirs(save_dir, exist_ok=True)


## Huấn luyện mô hình

### Cấu trúc huấn luyện:
1. Bật chế độ `train()` → cho phép tính gradient.  
2. Duyệt qua từng batch:
   - Dữ liệu đưa lên GPU (nếu có).  
   - Tính **logits**, **loss** và thực hiện **lan truyền ngược (backpropagation)**.  
3. Sau mỗi epoch:
   - Tính loss trung bình.  
   - Gọi `evaluate_model()` để đánh giá trên tập validation.  
   - Cập nhật `scheduler`.  
   - Lưu checkpoint nếu **AUC tốt nhất** được cải thiện.

### Lưu ý:
- Việc lưu checkpoint cho phép dừng huấn luyện giữa chừng mà không mất tiến trình.


In [10]:
for epoch in range(2):
    model.train()
    running_loss = 0.0
    for xb, yb in tqdm(train_loader):
        xb, yb = xb.to(device), yb.float().to(device)
        logits = model(xb).squeeze(-1)  # [batch]
        loss = criterion(logits, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * xb.size(0)

    train_loss = running_loss / len(train_loader.dataset)
    val_acc, val_auc, val_report = evaluate_model(model, val_loader)
    scheduler.step(val_loss := train_loss)  # or use val_auc etc
    print(f"[Epoch {epoch}] train_loss={train_loss:.4f} val_acc={val_acc:.4f} val_auc={val_auc:.4f} \n {val_report}")

    if val_auc > best_auc:
        best_auc = val_auc
        ckpt = os.path.join(save_dir, "tb_resnet50_best.pt")
        torch.save({"model_state": model.state_dict(), "class_to_idx": class_to_idx}, ckpt)
        print(f"  Saved best checkpoint to {ckpt}")


100%|██████████| 210/210 [17:44<00:00,  5.07s/it]


[Epoch 0] train_loss=0.0178 val_acc=1.0000 val_auc=nan 
               precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000       840

    accuracy                         1.0000       840
   macro avg     1.0000    1.0000    1.0000       840
weighted avg     1.0000    1.0000    1.0000       840



100%|██████████| 210/210 [17:10<00:00,  4.91s/it]


[Epoch 1] train_loss=0.0003 val_acc=1.0000 val_auc=nan 
               precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000       840

    accuracy                         1.0000       840
   macro avg     1.0000    1.0000    1.0000       840
weighted avg     1.0000    1.0000    1.0000       840



## Đánh giá mô hình sau huấn luyện

- Sau khi huấn luyện xong, ta đánh giá mô hình trên tập validation.  
- Các chỉ số gồm:
  - **Accuracy**: tỉ lệ dự đoán đúng tổng thể  
  - **AUC**: khả năng phân biệt giữa hai lớp (càng gần 1 càng tốt)  
  - **Classification Report**: hiển thị Precision, Recall, F1-score  

Kết quả giúp ta đánh giá xem mô hình đã nhận biết được đặc trưng của bệnh lao phổi hay chưa.


In [11]:
  # final eval
test_acc, test_auc, test_report = evaluate_model(model, val_loader)
print(f"Final val acc={test_acc:.4f}, auc={test_auc:.4f} \n {test_report}")

Final val acc=1.0000, auc=nan 
               precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000       840

    accuracy                         1.0000       840
   macro avg     1.0000    1.0000    1.0000       840
weighted avg     1.0000    1.0000    1.0000       840

